In [2]:
from MySQLdb import connect
from dask.distributed import Client
import dask.dataframe as dd
import witxcraft as wc
import pandas as pd

SINGLESTORE_HOST = "172.17.0.1"
SINGLESTORE_PORT = 3306

SINGLESTORE_URI = "mysql+mysqldb://root:root@{}:{}/demo_big".format(SINGLESTORE_HOST, SINGLESTORE_PORT)

In [3]:
client = Client(n_workers=8)
df = dd.read_sql_table('comments_small', SINGLESTORE_URI, npartitions=8, index_col='id')
df.head()

distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-sjqqj_jh', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-73v6f35x', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-ddae0x9c', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-ov_xucun', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-tz2ilguq', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-7hxs2u8k', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/exa

,text,creation_date,score
id,,,
85668560,@paulrevere because border-bottom added below ...,2018-03-17 07:11:47,0
85671232,You might want to choose between C and C++.,2018-03-17 10:19:53,0
85471418,I am sending it directly from the URL bar. :),2018-03-12 11:33:27,0
85471960,"You're moving the file to a folder called ""upl...",2018-03-12 11:48:28,0
85474543,Did you make a change in either resultGetChoic...,2018-03-12 12:52:23,0


In [4]:
len(df)

1000

In [5]:
ml = wc.fromwasmmod('example_wasm.wasm', witx='iface.witx')
ml.sentiment('covid is bad')

compound   -0.542326
positive    0.000000
negative    0.636364
neutral     0.363636
dtype: float64

In [6]:
out = df['text'].apply(ml.sentiment, meta=ml.sentiment.ret_meta)
out.compute()

,compound,positive,negative,neutral
id,,,,
85673569,-0.507974,0.000000,0.190002,0.809998
85559717,-0.367150,0.055316,0.140091,0.804593
85472668,-0.077228,0.000000,0.047619,0.952381
85473382,-0.240590,0.000000,0.000000,1.000000
85599569,0.000000,0.000000,0.000000,1.000000
...,...,...,...,...
116577834,0.273213,0.344262,0.000000,0.655738
116122333,0.000000,0.000000,0.000000,1.000000
115114681,-0.455198,0.000000,0.119295,0.880705


In [7]:
from pymysql import connect, cursors
s2 = connect(host=SINGLESTORE_HOST, port=SINGLESTORE_PORT, user='root', passwd='root', db='demo_big', local_infile=True, cursorclass=cursors.DictCursor)
cursor = s2.cursor()

In [8]:
cursor.execute("select count(*) from comments")
cursor.fetchall()

[{'count(*)': 82037742}]

In [9]:
cursor.execute("""
    create or replace function sentiment_vec(input varchar(256) not null)
    returns table(
        compound double,
        positive double,
        negative double,
        neutral double
    ) as wasm infile '/workspaces/wasi-data-python/examples/example_wasm.wasm';
""")

1

In [43]:
cursor.execute("select * from sentiment_vec('hello wasm day! I love wasm!')")
cursor.fetchall()

[{'compound': 0.6988421622775413,
  'positive': 0.5446265938069217,
  'negative': 0.0,
  'neutral': 0.45537340619307837}]

In [44]:
pd.read_sql("select c.text, c.score, s.* from (select * from comments where score > 10) as c, sentiment_vec(c.text) as s limit 10", con=SINGLESTORE_URI)

,text,score,compound,positive,negative,neutral
0,Thanks `npm install --legacy-peer-deps` works...,24,0.440434,0.325843,0.000000,0.674157
1,"Before voting to close, read meta question: ht...",13,0.000000,0.000000,0.000000,1.000000
2,Just a reminder: dns here is the name of a ser...,14,0.000000,0.000000,0.000000,1.000000
3,"It's correctness, not pedantry, and not a matt...",13,-0.019103,0.000000,0.059422,0.940578
4,I just added a `touch path/to/file` in the `Do...,24,-0.187672,0.000000,0.092850,0.907150
5,"For backward compatibility, it would be better...",11,0.440434,0.096990,0.000000,0.903010
6,Use `tf.global_variables_initializer` instead ...,16,0.000000,0.000000,0.000000,1.000000
7,`curl https://docs.oracle.com/javase/8/docs/ap...,48,-0.545074,0.069683,0.133934,0.796382
8,Because IEEE 754 formats include NAN in their ...,15,0.401924,0.144385,0.000000,0.855615
9,Doesn't work in a fresh markdown cell in jupyt...,22,-0.282824,0.000000,0.078918,0.921082


In [94]:

df = pd.read_sql("""
    select
        (score - (score %% 10)) as score_bucket,
        count(*) as num_scores,
        abs(min(s.compound)) as "negative",
        avg(s.compound) as "avg",
        max(s.compound) as "positive"
    from
        (select * from comments where score >= 10) as c,
        sentiment_vec(c.text) as s
    group by 1
    having positive > 0 and negative > 0
    order by 1 asc
""", con=SINGLESTORE_URI)

df.head()

,score_bucket,num_scores,negative,avg,positive
0,10,115285,0.999408,0.117165,0.988480
1,20,27409,0.988882,0.122059,0.989240
2,30,12064,0.982151,0.121961,0.987701
3,40,6701,0.984391,0.125527,0.987944
4,50,4130,0.957839,0.128716,0.977145


In [115]:
import plotly.express as px
import plotly.graph_objects as go
import statsmodels

colorscheme = ["#00E676" ,"#AA00FF"]

# ols = Ordinary Least Squares
fig = px.scatter(df,
    x="score_bucket",
    y=["positive", "negative"],
    trendline="ols",
    color_discrete_sequence=colorscheme,
    template="plotly_dark",
    labels={
        "score_bucket": "Comment Score",
        "value": "sentiment",
    }
)

fig.update_layout(
    paper_bgcolor='#1B1A21',
    plot_bgcolor='#1B1A21',
)

fig.show()